## Merge Bus Usage Data & Bus Inform Data to Total Data Frame

In [61]:
import pandas as pd
import pickle
import warnings
warnings.filterwarnings("ignore")

In [62]:
## Bus Base Data Frame
busroute = pd.read_csv(
    filepath_or_buffer="../data/버스기본데이터/gyeongn_busroute_total.csv"
    )
usecols = ['city_name', 'routeno', 'routetp', 'startnodenm', 'endnodenm', 'startvehicletime', 'endvehicletime', 'intervaltime', 'intervalsattime', 'intervalsuntime']
city_names = busroute["city_name"].unique()
busroute = busroute[usecols]

## Merge Bus Usage Data & Bus Inform Data to Total Data Frame
total_busdata = pd.DataFrame()

for city_name in city_names:
    ## Load Bus Usage data
    with open(f"../data/버스이용데이터/{city_name}_usage.pkl", "rb") as f:
        bususage_ = pickle.load(f)
        
    bususage_ = pd.DataFrame(bususage_).reset_index()
    bususage_ = bususage_.rename(columns={
        "index" : "routeno", 
        f"{city_name}" : "usage"
        })

    ## Load Bus Inform data
    with open(f"../data/버스정보데이터/{city_name}_inform.pkl", "rb") as f:
        businform_ = pickle.load(f)

    businform_ = pd.DataFrame(businform_[city_name]).T.reset_index()
    businform_ = businform_.rename(columns={
        "index" : "routeno"
    })

    ## Merge with routeno column
    busroute_ = busroute[busroute["city_name"] == city_name]
    busroute_ = pd.merge(busroute_, bususage_, how="left", on="routeno")
    busroute_ = pd.merge(busroute_, businform_, how="left", on="routeno")
    
    total_busdata = pd.concat([total_busdata, busroute_], axis=0).reset_index(drop=True)

In [63]:
total_busdata

,city_name,routeno,routetp,startnodenm,endnodenm,startvehicletime,endvehicletime,intervaltime,intervalsattime,intervalsuntime,usage,노선정류장수,노선거리(km),굴곡도
0,창원시,1,마을버스,신전,창원역,540.0,2210.0,40.0,40.0,40.0,33931.0,105.0,55.9,2.0
1,창원시,2,마을버스,유등,창원역,725.0,2150.0,170.0,170.0,170.0,2558.0,83.0,53.5,1.7
2,창원시,3,마을버스,상리,진영시외주차장,750.0,1655.0,220.0,220.0,220.0,299.0,74.0,43.8,2.6
3,창원시,7,마을버스,창원역,자여입구,620.0,2310.0,11.0,15.0,15.0,63612.0,35.0,21.1,1.0
4,창원시,8,마을버스,창원역,삼성창원병원종점(8),600.0,2240.0,17.0,17.0,17.0,68306.0,39.0,8.8,1.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2508,합천군,660-1,농어촌(일반)버스,합천터미널,합천터미널,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None
2509,합천군,670-2,농어촌(일반)버스,합천터미널,합천터미널,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None
2510,합천군,810-3,농어촌(일반)버스,부림면정류장,합천터미널,NaN,NaN,NaN,NaN,NaN,45.0,29.0,25.3,1.3
2511,합천군,110-13,농어촌(일반)버스,부림면정류장,합천터미널,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None


In [64]:
# total_busdata.to_csv("../data/total_busdata.csv", index=False)